In [ ]:
# Install esm and other dependencies
! pip install boto3
# ! pip install esm
! pip install esm==3.1.1
! pip install py3Dmol
! pip install matplotlib
! pip install dna-features-viewer

In [ ]:
# Uncomment to enable boto3 debug logging.

# import boto3
# import logging
# logging.basicConfig(level=logging.DEBUG)
# boto3.set_stream_logger(name="botocore")

In [ ]:
import boto3

AWS_REGION='us-east-2'
AWS_PROFILE='partner-prem-dev'

boto3.setup_default_session(region_name=AWS_REGION, profile_name=AWS_PROFILE)

In [ ]:
# CLI output:

# $ terraform output
# sagemaker_endpoints = {
#   "exploratory_model" = {
#     "endpoint_config_name" = "EndpointConfig-Endpoint-ESMC-300M-1-f1eddb86"
#     "endpoint_name" = "Endpoint-ESMC-300M-1-f1eddb86"
#     "endpoint_url" = "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/Endpoint-ESMC-300M-1-f1eddb86/invocations"
#     "forge_model_name" = "esmc-300m-2024-12"
#   }
# }

# Console: https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/endpoints
SAGE_ENDPOINT_NAME = "Endpoint-ESMC-300M-1"
SAGE_ENDPOINT_NAME = "Endpoint-ESMC-300M-1-f1eddb86"

# EvolutionaryScale/Forge Model name (not SageMaker Model Name)
# Update from the Forge Console: https://forge.evolutionaryscale.ai/console
# Options: esm3-open-2024-03, esmc-300m-2024-12, esmc-600m-2024-12
MODEL_NAME = "esmc-300m-2024-12"

In [ ]:
from esm.sdk.sagemaker import ESM3SageMakerClient
from esm.sdk.api import ESMProtein, LogitsConfig

sagemaker_client = ESM3SageMakerClient(
   # E.g. "Endpoint-ESMC-6B-1"
   endpoint_name=SAGE_ENDPOINT_NAME,
   # E.g. "esmc-6b-2024-12". Same model names as in Forge.
   model=MODEL_NAME
)

protein = ESMProtein(sequence="AAAAA")
protein_tensor = sagemaker_client.encode(protein)
logits_output = sagemaker_client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings)